In [22]:
import csv
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads

In [144]:
def close_line_string(wkt_str):
    
    str_polygons = [poly.replace('(','') for poly in re.findall(r'\((.*?)\)', wkt_str)]

    polygons=[]
    for str_polygon in str_polygons:
        coordinates=[]
        for coordinate in str_polygon.split(','):
            coordinates.append(coordinate.strip())
        polygons.append(coordinates)
    
    # Check if the polygon is not closed
    
    res = []
    for polygon in polygons:
        if polygon[0]!=polygon[-1]:
            polygon += [polygon[0]]
            res.append(polygon)
        else:
            res.append(polygon)

    return f"POLYGON {res}".replace("'",'').replace('[','(').replace(']',')')

In [173]:
df = pd.read_csv('csv/Prueba_programacion.csv', delimiter=';')
df = df[:20]

In [174]:
df['GEOMETRIA'] = df['GEOMETRIA'].apply(close_line_string)

In [181]:
gdf = gpd.GeoDataFrame(df)
gdf['geometry'] = gdf['GEOMETRIA'].apply(loads)
gdf = gdf.set_crs('EPSG:4326')

In [184]:
gdf.to_file(driver='ESRI Shapefile', filename='shp/polygons.shp')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
